# Color dithering

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np

import astro3d
from astro3d.image_stack import color_replace

Make up some fake data: two gaussian bumps

In [ ]:
x = np.linspace(-2, 2, 1000)
y = np.linspace(-2, 2, 990)
X,Y = np.meshgrid(x, y, indexing='ij')

# peak density
rho0 = 1e0
dyn_range = 1e-3

data = rho0 * (
    1.0 * np.exp(- ((X-0.5)**2 + (Y-0.5)**2)/(2*0.5**2)) +
    0.1 * np.exp(- ((X+0.5)**2 + (Y+0.5)**2)/(2*0.5**2))
    )

# Define norm and plot the data

In [ ]:
vmax = data.max()
norm = LogNorm(dyn_range * vmax, vmax)

# plot the data

f, ax = plt.subplots()
ax.set_aspect('equal')
cc=ax.pcolormesh(x, y, data.T, norm=norm)
plt.colorbar(cc);

Define three densities we want to color, along with the scatter around that density value in dex. Fill is the filling factor to scale some colors up or down.

In [ ]:
#rho_i = [1e-2 * vmax, 1e-1 * vmax, vmax]
rho_i = norm.inverse([0.15, 0.5, 0.8]).data
sig = [0.3, 0.1, 0.3]
fill = [1.0, 1.0, 1.0]

Show a histogram of the data values

In [ ]:
bins = np.geomspace(dyn_range * vmax, vmax, 100)
counts, _ = np.histogram(data.ravel(), bins=bins)

In [ ]:
f, ax = plt.subplots()
ax.bar(bins[:-1], counts, align='edge', width=np.diff(bins))
ax.set_xscale('log')
ax.set_yscale('log')
ax2 = ax.secondary_xaxis('top', functions=(norm, norm.inverse))
ax2.set_xscale('linear')
for _rho, _sig in zip(rho_i, sig):
    ax.axvline(_rho, c='k', ls='--')
    ax.errorbar(_rho, 10.**np.mean(np.log10(ax.get_ylim())),
                xerr=[
                    [_rho - norm.inverse(norm(_rho) - _sig)],
                    [norm.inverse(norm(_rho) + _sig) - _rho]], c='k', capsize=5)

for each color, we compute the distance-image, i.e. how far away we are from the desired contour.

We use the normalized squared distance in log space for each pixel value $D_{k,l}$. Each color, which is specified by its density threshold $\rho_i$ and the width around that density value in dex, $\sigma_i$
$$
d^2_{k,l,i} = \frac{\left(\log_{10}(\rho_i / D_{k, l})\right)^2}{\sigma_i^2}
$$

The amount of color is then assigned according to that distance:

$$\mathrm{color}_i = \frac{1}{1 + d^2 _ {k,l,i}}$$

In [ ]:
dist_sq = (np.log10(np.array(rho_i)[None, None, :] / data[..., None]) / sig)**2
color_density = 1 / (1 + dist_sq) * fill

In [ ]:
f, ax = plt.subplots(1, 4, dpi=100, figsize=(16, 4))
ax[-1].imshow(color_density, origin='lower', vmin=0, vmax=1)
ax[-1].set_title('un-dithered')

for i, _fill in enumerate([0.1, 0.5, 1]):
    _color_density = 1 / (1 + dist_sq) * _fill

    im_cd = astro3d.fmodule.dither_colors(_color_density)
    ax[i].imshow(im_cd, origin='lower', vmin=0, vmax=1)
    ax[i].set_title(f'fill = {_fill:.2g}')

In [ ]:
f, axs = plt.subplots(1, 4, figsize=(12, 3), dpi=150)
args = {'origin':'lower', 'cmap':'gray_r'}

axs[0].imshow(im_cd[:, :, 0], **args)
axs[1].imshow(im_cd[:, :, 1], **args)
axs[2].imshow(im_cd[:, :, 2], **args)
axs[3].imshow(im_cd.sum(-1), **args)

print(f'there is {"no" if im_cd.max(-1).max()==1 else ""} overlap')

# Replacing Colors

So far we worked with 3 unique RGB colors. For the printing, we have, however only the 3 base-colors VeroCyan, VeroMagenta, and VeroYellow available. We can either use those or we could try to mix them to another color, for example by replacing 50% of the currently red pixels with Magenta, and 50% with Yellow. The mixing on the Stratasys Pallete between C, M, and Y looks different from just a linear interpolation.

In [ ]:
VeroT_sRGB = np.array([255, 255, 255]) / 255
VeroC_sRGB = np.array([29,  85,  111]) / 255
VeroM_sRGB = np.array([149, 39,  87])  / 255
VeroY_sRGB = np.array([192, 183, 52])  / 255

## Simple case:
replacing colors direclty (doesn't really make a difference as the material is assigned during printing by hand).

In [ ]:
im = im_cd.copy()
im = color_replace(im, [1,0,0], VeroM_sRGB)
im = color_replace(im, [0,1,0], VeroY_sRGB)
im = color_replace(im, [0,0,1], VeroC_sRGB)
im = color_replace(im, [0,0,0], VeroT_sRGB)

#check that we didn't mess up the colors
_cols = np.unique(im.reshape(-1, 3), axis=0)
print(f'there are {len(_cols)} color(s) in the image')
f, ax = plt.subplots()
ax.imshow([_cols])
ax.axis('off')

f, ax = plt.subplots(1,2, dpi=150)
ax[0].imshow(im_cd)
ax[1].imshow(im);

## Interesting case:
replacing one color by a color mix

In [ ]:
im = im_cd.copy()
im = color_replace(im, [1,0,0], [VeroY_sRGB, VeroC_sRGB], f=[0.2, 0.8])
im = color_replace(im, [0,1,0], [VeroM_sRGB, VeroC_sRGB], f=[0.2, 0.8])
im = color_replace(im, [0,0,1], [VeroM_sRGB, VeroY_sRGB], f=[0.5, 0.5])
im = color_replace(im, [0,0,0], VeroT_sRGB)

# check that we didn't mess up the colors
_cols = np.unique(im.reshape(-1, 3), axis=0)
print(f'there are {len(_cols)} color(s) in the image')
f, ax = plt.subplots()
ax.imshow([_cols])
ax.axis('off');

f, ax1 = plt.subplots(1,2, dpi=150)
ax1[0].imshow(im_cd)
ax1[1].imshow(im)